In [1]:
print("ok")

ok


In [2]:
%pwd

'g:\\Projects\\LLM_projects\\Medical_chot_bot\\research'

In [3]:
import os

os.chdir("../")

In [4]:
%pwd

'g:\\Projects\\LLM_projects\\Medical_chot_bot'

In [5]:
from langchain.document_loaders import DirectoryLoader,PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [7]:
extracted_data=load_pdf_file(data='Data/')

In [10]:
#extracted_data

In [8]:
def text_splitter(extracted_data):
    splitter= RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    chunks=splitter.split_documents(extracted_data)
    return chunks

In [24]:
chunks = text_splitter(extracted_data)
print("length of chunks",len(chunks))



length of chunks 5860


In [10]:
from langchain.embeddings import HuggingFaceEmbeddings

#384 dim

In [11]:
def download_huggingface_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [12]:
embeddings = download_huggingface_embeddings()

C:\Users\shaik\AppData\Local\Temp\ipykernel_24616\515697783.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
g:\Projects\LLM_projects\Medical_chot_bot\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
query = embeddings.embed_query("What is the impact of COVID-19 on the economy?")
print("length",len(query))

length 384


In [14]:
from dotenv import load_dotenv

load_dotenv()

True

In [20]:
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")

In [21]:
from pinecone import Pinecone
from pinecone import ServerlessSpec

In [22]:
pc = Pinecone(api_key=PINECONE_API_KEY)    

index_name = "medicat-chat-bot"

In [23]:
pc.create_index(name=index_name,
                dimension=384,
                metric="cosine",
                spec=ServerlessSpec
                (
                    cloud="aws",
                    region="us-east-1"
                    )
)

In [31]:
import os 

os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

In [32]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=chunks,
    embedding=embeddings,
    index_name=index_name
)

ProtocolError: ('Connection aborted.', ConnectionAbortedError(10053, 'An established connection was aborted by the software in your host machine', None, 10053, None))

In [33]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [34]:
retriver = docsearch.as_retriever(search_type="similarity",search_kwargs={"k":3})

In [ ]:
retrieved_docs = retriver.invoke("What is Acne?")
retrieved_docs

[Document(id='5dd1d247-a6ae-49c8-afc3-9482a081ce0d', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'), Document(id='397a6853-04c1-4706-97ef-44f51ca9a2bd', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 38.0, 'page_label': '39', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed.(Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM - 

In [36]:
retrieved_docs

[Document(id='5dd1d247-a6ae-49c8-afc3-9482a081ce0d', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='397a6853-04c1-4706-97ef-44f51ca9a2bd', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 38.0, 'page_label': '39', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed.(Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM -

In [37]:
GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")

In [38]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro",temperature=0.4,max_tokens=100,api_key=GOOGLE_API_KEY)


In [43]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    """ You are an assistant for question-answering tasks. 
    Use the following pieces of retrieved context to answer 
    the question. If you don't know the answer, say that you 
    don't know. Use three sentences maximum and keep the 
    answer concise."
    
    {context}"""
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [44]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriver, question_answer_chain)

In [45]:
response = rag_chain.invoke({"input": "what is Acne?"})
print(response["answer"])

Acne, medically known as acne vulgaris, is a common skin disease.  It's characterized by pimples forming on the face, chest, and back due to clogged pores. These clogged pores are a result of oil, dead skin cells, and bacteria.


In [46]:
response = rag_chain.invoke({"input": "what is stats?"})
print(response["answer"])

This question cannot be answered from the given context.  The provided text discusses blood counts and athletic heart syndrome, not the meaning or definition of "stats".
